## RAG란?
- 자연어 처리(NLP) 분야의 혁신적인 기술로, 기존 모델의 한계를 넘어서 정보 검색과 생성을 통합하는 방법론
- 풍부한 정보를 담고 있는 대규모 문서 데이터베이스에서 관련정보를 검색하고, 
- 이를 통해 언어 모델이 더 정확하고 상세한 답변을 생성 할 수 있습니다.

### 핵심 개념
- **검색(Retrieval)**: 질문과 관련된 문서나 정보를 벡터 유사도 기반으로 찾기
- **증강(Augmented)**: 찾은 정보를 언어 모델의 컨텍스트에 추가
- **생성(Generation)**: 증강된 정보를 바탕으로 더 정확하고 신뢰할 수 있는 답변 생성


### RAG의 장점
1. 최신 정보 반영 가능 (지식 갱신이 간편)
2. 출처 기반 응답 제공
3. 도메인 특화 응답 가능

### RAG의 기본구조

![RAG.png](./images/RAG.png)



**RAG-FLOW**
![RAG_Flow.png](./images/RAG_Flow.png)


## PDF를 문서로 사용해서 RAG 시스템 구축하기
- 다음 실습 예시는 기본적인 RAG 구조를 구현합니다.
- 추후, 개발을 진행할때는 각각의 구현 상황에 맞는 모듈로 바꾸는 것 만으로도 여러분들만의 RAG 시스템을 구축 할 수 있습니다.

- 예시  
지금 예시에서는 PDF를 읽어와 문서를 저장하고 있지만, 제공되는 문서가 Text일 경우 Docuemnt Loader 모듈을 바꾸어 사용할 수 있습니다.  

**실습자료**
- 제목 : AI.GOV 해외동향 2025-1호
- 링크 : [한국지능사회정보진흥원](https://www.innovation.go.kr/ucms/bbs/B0000051/view.do?nttId=17774&menuNo=300145&pageIndex=)  
- 출처 : NIA 한국지증정보원원

*링크에서 파일을 다운로드 받은후 최상단 data 폴더에 위치시켜주세요*

### Step 1 : 문서 로드(Document Load)

In [ ]:
# langchain_community : Langchain 핵심 기능 외에 커뮤니티가 기여하고 유지보수하는 외부 서비스 패키지지
%pip install langchain_community

In [ ]:
# python으로 PDF문서를 빠르고 쉽게 열고 내용을 추출하거나 파일을 수정 및 이미지 저장을 해주는 라이브러리
%pip install pypdf

In [ ]:
# 1. DoucmentLoader
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/[AI.GOV_해외동향]_2025-1호.pdf")

docs = loader.load()

print(f"1번째 문서 내용 : {docs[0].page_content}")
print(f"총 문서의 갯수 : {len(docs)}")

메타데이터 확인해보기

In [ ]:
print(docs[0].__dict__)

### Step 2 : 문서 분할하기 (Text Splitter)

### Character Text Splitter
`CharacterTextSplitter`는 가장 기본적인 텍스트 분할 도구입니다.  
특정 문자(separator)를 기준으로 텍스트를 나누어 작은 청크(chunk)로 만들어줍니다.

**chunk란?**  
문장을 분석/처리 하기 쉽게 텍스트를 작은 단위로 나눈 조각을 의미합니다.

기본 사용법

CharacterTextSplitter의 기본 구조는 다음과 같습니다:

**주요 매개변수**
- `separator`: 텍스트를 나눌 기준 문자 (기본값: `"\n\n"`)
- `chunk_size`: 각 청크의 최대 문자 수
- `chunk_overlap`: 인접한 청크 간 겹치는 문자 수

Chunk Overlap이해하기

In [ ]:
# Chunck Overlap 이해하기
from langchain_text_splitters import CharacterTextSplitter

sample_text = "이것은 오버랩 개념을 설명하기 위한 예시 문장입니다. 문맥이 끊어지지 않도록 도와줍니다."

# 1. Overlap 없이 분할하는 Splitter 생성
no_overlap_splitter = CharacterTextSplitter(
    separator=" ",         # 띄어쓰기 단위로 분할
    chunk_size=10,         # Chunk 최대 크기를 25자로 설정
    chunk_overlap=5,       # 겹치는 부분 없음
)

# 2. 텍스트 분할 실행
chunks = no_overlap_splitter.split_text(sample_text)

In [ ]:
print("--- Overlap이 없을 때 (chunk_overlap=0) ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: \"{chunk}\"")

In [ ]:
print("--- Overlap이 있을 때 (chunk_overlap=10) ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: \"{chunk}\"")

띄워쓰기가 없는 아주 긴 단어라면?

In [ ]:
long_word_text = "이것은테스트입니다슈퍼울트라하이퍼메가캡숑단어"

# chunk_size를 10으로 아주 작게 설정
long_word_splitter = CharacterTextSplitter(
    # separator=" ",         # 띄어쓰기 단위로 분할
    chunk_size=10,         # Chunk 최대 크기를 10자로 설정
    chunk_overlap=0,       # 겹치는 부분 없음
)

chunks = long_word_splitter.split_text(long_word_text)

print(f"--- chunk_size=10 설정 결과 ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} (길이: {len(chunk)}): \"{chunk}\"")

### 작동 우선순위 (Hierarchy)
- 코드는 설정된 separators 리스트의 왼쪽부터 순서대로 적용하여 자를 위치를 찾습니다.

```Python
separators=["\n\n", "\n", ". ", " "]
```
1순위: 문단 (\n\n)
- 가장 먼저 전체 문단을 통째로 담으려 시도합니다.  
(설정된 크기를 넘지 않으면 문단 전체가 하나의 청크가 됨)  
  
2순위: 줄바꿈 (\n)  
- 문단이 너무 길면, 줄바꿈 단위로 쪼갭니다.  
  
3순위: 문장 (. )  
- 그래도 길면, 마침표를 기준으로 문장 단위로 쪼갭니다.
  
4순위: 단어/공백 ( )  
- 문장마저 너무 길면, 띄어쓰기 단위로 쪼갭니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # 바꿔써보기

# 2. 문서 분할 
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "], # 기본값
    chunk_size=200, # 최대 크기를 설정
    chunk_overlap=20 
)

test_text = """인공지능 기술이 발전함에 따라 거대언어모델(LLM)의 활용도가 높아지고 있습니다. LLM은 방대한 데이터를 학습하여 자연스러운 텍스트를 생성하는 데 탁월한 능력을 보입니다.

하지만 모델이 학습하지 않은 최신 정보나 기업 내부의 비공개 데이터에 대해서는 답변하지 못하거나, 사실이 아닌 내용을 사실인 것처럼 말하는 환각(Hallucination) 현상이 발생할 수 있습니다. 이러한 한계를 극복하기 위해 등장한 기술이 바로 검색 증강 생성(RAG)입니다.

RAG는 사용자의 질문이 들어오면 먼저 외부 데이터베이스에서 관련 문서를 검색합니다. 그 후 검색된 정확한 정보를 프롬프트에 포함하여 LLM이 답변을 생성하도록 유도합니다. 이 과정을 통해 AI는 더욱 신뢰성 있고 구체적인 답변을 제공할 수 있게 됩니다."""

chunks = recursive_splitter.split_text(test_text)

print(f"--- chunk_size=10 설정 결과 ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} (길이: {len(chunk)}): \"{chunk}\"")

--- chunk_size=10 설정 결과 ---
Chunk 1 (길이: 96): "인공지능 기술이 발전함에 따라 거대언어모델(LLM)의 활용도가 높아지고 있습니다. LLM은 방대한 데이터를 학습하여 자연스러운 텍스트를 생성하는 데 탁월한 능력을 보입니다."
Chunk 2 (길이: 156): "하지만 모델이 학습하지 않은 최신 정보나 기업 내부의 비공개 데이터에 대해서는 답변하지 못하거나, 사실이 아닌 내용을 사실인 것처럼 말하는 환각(Hallucination) 현상이 발생할 수 있습니다. 이러한 한계를 극복하기 위해 등장한 기술이 바로 검색 증강 생성(RAG)입니다."
Chunk 3 (길이: 142): "RAG는 사용자의 질문이 들어오면 먼저 외부 데이터베이스에서 관련 문서를 검색합니다. 그 후 검색된 정확한 정보를 프롬프트에 포함하여 LLM이 답변을 생성하도록 유도합니다. 이 과정을 통해 AI는 더욱 신뢰성 있고 구체적인 답변을 제공할 수 있게 됩니다."


다시 RAG 구성으로

In [ ]:
# 2. TextSplitter
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정으로 텍스트 분할기 생성
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,     # 각 청크 최대 200자
    chunk_overlap=20   # 20자씩 겹침
)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수: {len(split_documents)}")

분할결과

In [ ]:
print(split_documents[2].page_content)

### Step 3 : 임베딩 준비하기 (Embedding)

#### Embedding이란?
**텍스트 → 수치 벡터(vector)** 로 변환하는 작업  
즉, 사람의 언어를 컴퓨터가 이해할 수 있는 형식으로 바꾸는 것.

`RAG`에서 관련있는 문서를 검색 해 올 수 있어야 하기 때문에, 문서를 벡터화 시켜, 의미가 비슷한 문서를 찾을 수 있다.

#### OpenAIEmbeddings
- OpenAIEmbeddings는 LangChain에서 OpenAI의 강력한 임베딩 모델을 사용하기 위한 도구입니다.  


**모델별 비교표**

| 모델명 | 비용 (1백만 토큰 당) | 기본 벡터 차원 | 특징 및 장단점 |
| :--- | :--- | :--- | :--- |
| **`text-embedding-3-small`** | **$0.02** | 1536 | **(추천)** **가성비가 가장 뛰어난 최신 모델.** `ada-002`보다 훨씬 저렴하지만 성능(특히 다국어)은 더 우수함. 대부분의 경우에 추천됨. |
| **`text-embedding-3-large`** | $0.13 | 3072 | **최고 성능 모델.** 가장 높은 정확도를 제공하며, 미묘하고 복잡한 의미를 파악하는 데 가장 강력함. 비용과 벡터 저장 공간이 더 필요함. |
| `text-embedding-ada-002` | $0.10 | 1536 | **(구세대 모델)** 과거에 가장 널리 쓰였던 모델. 특별한 이유가 없다면 이제는 `3-small` 모델 사용이 모든 면에서 유리함. |


In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

In [ ]:
# 3. Embedding
from langchain_openai import OpenAIEmbeddings

# 1. OpenAI 임베딩 모델 초기화 (ChatGPT-4o-mini와 호환)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # OpenAI의 최신 임베딩 모델
    # API 키는 환경변수 OPENAI_API_KEY에서 자동으로 읽어옵니다
)

print(f"모델명: {embeddings.model}")

임베딩 확인해보기

In [ ]:
document_vectors = embeddings.embed_documents(chunks)

print(f"--- 문서 임베딩 결과 ---")
print(f"총 {len(document_vectors)}개의 Chunk가 벡터로 변환되었습니다.")
print(f"각 벡터의 차원(길이): {len(document_vectors[0])}")
print(f"첫 번째 Chunk의 벡터(일부): {document_vectors[0]}...")

### Step 4 : Vector Store 생성 및 문서 임베딩

In [ ]:
%pip install faiss-cpu

In [ ]:
# 4. VectorStore
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=split_documents, embedding=embeddings)

검색해보기

In [ ]:
query = "프랑스 AI Action Summit 투자 유치액"

# 가장 유사한 문서를 찾음
# k : 찾아올 갯수
docs = db.similarity_search(query, k=2)

print("[가장 유사한 문서]\n" + docs[0].page_content)
print("[그다음 유사한 문서]\n" + docs[1].page_content)

### Step 5 : Retriever 생성

In [ ]:
# 5. Retriever
retriever = db.as_retriever()

### Step 6 : 프롬프트 생성

In [ ]:
# 6. Prompt
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문에 답변하는 작업을 수행하는 어시스턴트입니다.
다음에 제공된 문맥 정보를 바탕으로 질문에 답하세요.
정답을 모를 경우, 모른다고만 말하세요.
답변은 반드시 한국어로 작성하세요.

#문맥:
{context}

#질문:
{question}

#답변:"""
)

### Step 7 : LLM 객체 생성

In [ ]:
# 7. Model
from langchain.chat_models import init_chat_model

# 문맥기반으로 대답을 해야하기 때문에 창의성 0으로 설정
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.0)

### Step 8 : 체인 생성

In [ ]:
# 8. Chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(), # 다음 체인으로 값을 그대로 넘김
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 9. 테스트
query = "주요 해외 AI 에이전트 서비스 동향에 대해 알려줘"
response = chain.invoke(query)

print(response)

In [ ]:
llm.invoke(query)